**Review**

Hi, my name is Dmitry and I will be reviewing your project.
  
You can find my comments in colored markdown cells:
  
<div class="alert alert-success">
  If everything is done successfully.
</div>
  
<div class="alert alert-warning">
  If I have some (optional) suggestions, or questions to think about, or general comments.
</div>
  
<div class="alert alert-danger">
  If a section requires some corrections. Work can't be accepted with red comments.
</div>
  
Please don't remove my comments, as it will make further review iterations much harder for me.
  
Feel free to reply to my comments or ask questions using the following template:
  
<div class="alert alert-info">
  For your comments and questions.
</div>
  
First of all, thank you for turning in the project! You did a pretty good job overall, but there are some issues that need to be fixed before the project can be accepted. Let me know if you have questions!

## Project Description
This project analyzes 3 regions of possible implementation of new oil wells for the mining company OilyGiant. The project consists of developing Machine Learning models to predict the productivity of each region, and thus evaluate the viability of implementing new oil wells.<br>
Taking into account the historical production dataframes of each region, we train several regression models using LinearRegression and RandomForest algorithms. We then evaluated their performance using the MSE, RMSE, MAE and R2 Score metrics. After the model analysis, we take into account the average cost of implementing new oil wells ($100M USD for 200 oil wells), and evaluate the profitability of each Region according to the predictions of each model. For this we will use the Bootstrapping technique, taking random samples of 500 points and selecting the 200 most productive points of each sample. Finally, we calculate the average production of each region within a 95% confidence interval, indicating the minimum and maximum intervals (2.5% and 97.5%).<br>

## Import Modules used in the project

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

## Load datasets, explore and prepare the data

In [3]:
# Create variables for the dataframes of each region
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

# Create function to print info for each dataset and look for null or repeated values
def print_info(df):
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns}\n")
    print(f"Dtypes:\n {df.dtypes}\n")
    print(f"Duplicated rows: {df.duplicated().sum()}")
    print(f"Null values:\n {df.isna().sum()}")
    display(df.head())
    print("="*60)

# Print info for each dataset
print("REGION A (data_0):\n")
print_info(data_0)
print("REGION B (data_1):\n")
print_info(data_1)
print("REGION C (data_2):\n")
print_info(data_2)

REGION A (data_0):

Shape: (100000, 5)
Columns: Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')

Dtypes:
 id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object

Duplicated rows: 0
Null values:
 id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


REGION B (data_1):

Shape: (100000, 5)
Columns: Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')

Dtypes:
 id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object

Duplicated rows: 0
Null values:
 id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


REGION C (data_2):

Shape: (100000, 5)
Columns: Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')

Dtypes:
 id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object

Duplicated rows: 0
Null values:
 id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Each dataset represent the Oil Wells parameters for each region. There are no duplicated or null values and the datasets are the same shape. The data seems to be clean and properly scaled for Machine Learning.<br>
We can proceed by dropping the **id** columns, since this is only a generic identifier for each row and is not relevant to the model. Then we will split the datasets in **features** (f0, f1, f2) and **target** (product) variables.

<div class="alert alert-success">
<b>Reviewer's comment</b>

The data was loaded and inspected

</div>

In [4]:
# Drop the features that are not relevant to the model
data_0.drop(columns=["id"], axis=1, inplace=True)
display(data_0.head())
data_1.drop(columns=["id"], axis=1, inplace=True)
display(data_1.head())
data_2.drop(columns=["id"], axis=1, inplace=True)
display(data_2.head())

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


<div class="alert alert-success">
<b>Reviewer's comment</b>

Good!

</div>

In [5]:
# Define 'target' and 'feature' variables for each dataset
print("REGION A - Target (data_target_0) and Features (data_features_0):\n","="*66)
data_target_0 = data_0['product']
data_features_0 = data_0.drop('product', axis=1)
display(data_target_0.head())
display(data_features_0.head())

print("REGION B - Target (data_target_1) and Features (data_features_1):\n","="*66)
data_target_1 = data_1['product']
data_features_1 = data_1.drop('product', axis=1)
display(data_target_1.head())
display(data_features_1.head())

print("REGION C - Target (data_target_2) and Features (data_features_2):\n","="*66)
data_target_2 = data_2['product']
data_features_2 = data_2.drop('product', axis=1)
display(data_target_2.head())
display(data_features_2.head())

REGION A - Target (data_target_0) and Features (data_features_0):


0    105.280062
1     73.037750
2     85.265647
3    168.620776
4    154.036647
Name: product, dtype: float64

,f0,f1,f2
0,0.705745,-0.497823,1.221170
1,1.334711,-0.340164,4.365080
2,1.022732,0.151990,1.419926
3,-0.032172,0.139033,2.978566
4,1.988431,0.155413,4.751769


REGION B - Target (data_target_1) and Features (data_features_1):


0      3.179103
1     26.953261
2    134.766305
3    137.945408
4    134.766305
Name: product, dtype: float64

,f0,f1,f2
0,-15.001348,-8.276000,-0.005876
1,14.272088,-3.475083,0.999183
2,6.263187,-5.948386,5.001160
3,-13.081196,-11.506057,4.999415
4,12.702195,-8.147433,5.004363


REGION C - Target (data_target_2) and Features (data_features_2):


0     27.758673
1     56.069697
2     62.871910
3    114.572842
4    149.600746
Name: product, dtype: float64

,f0,f1,f2
0,-1.146987,0.963328,-0.828965
1,0.262778,0.269839,-2.530187
2,0.194587,0.289035,-5.586433
3,2.236060,-0.553760,0.930038
4,-0.515993,1.716266,5.899011


In [6]:
# Split each dataset in Training and Validation Sets (75:25 ratio)
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    data_features_0, data_target_0, test_size=0.25, random_state=12345)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    data_features_1, data_target_1, test_size=0.25, random_state=12345)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    data_features_2, data_target_2, test_size=0.25, random_state=12345)

print("Shapes of each Training and Validation set:\n","="*45)
print("REGION A:")
print(f"Training Set (Features:{features_train_0.shape}, Target:{target_train_0.shape}")
print(f"Validation Set (Features:{features_valid_0.shape}, Target:{target_valid_0.shape}")
print()
print("REGION B:")
print(f"Training Set (Features:{features_train_1.shape}, Target:{target_train_1.shape}")
print(f"Validation Set (Features:{features_valid_1.shape}, Target:{target_valid_1.shape}")
print()
print("REGION C:")
print(f"Training Set (Features:{features_train_2.shape}, Target:{target_train_2.shape}")
print(f"Validation Set (Features:{features_valid_2.shape}, Target:{target_valid_2.shape}")

Shapes of each Training and Validation set:
REGION A:
Training Set (Features:(75000, 3), Target:(75000,)
Validation Set (Features:(25000, 3), Target:(25000,)

REGION B:
Training Set (Features:(75000, 3), Target:(75000,)
Validation Set (Features:(25000, 3), Target:(25000,)

REGION C:
Training Set (Features:(75000, 3), Target:(75000,)
Validation Set (Features:(25000, 3), Target:(25000,)


The datasets are ready to train, validate and compare Machine Learning models.<br>

<div class="alert alert-success">
<b>Reviewer's comment</b>

The data for each region was split into train and validation

</div>

## Train, test and compare Machine Learning models for each Region.
Since this is a **Regression** task, we will use the **Linear Regression** model from scikit learn.

In [7]:
# Train a Linear Regression model for each region and define variables for prediction.
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predicted_train_0 = pd.Series(model_0.predict(features_train_0))
predicted_valid_0 = pd.Series(model_0.predict(features_valid_0))

model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predicted_train_1 = pd.Series(model_1.predict(features_train_1))
predicted_valid_1 = pd.Series(model_1.predict(features_valid_1))

model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predicted_train_2 = pd.Series(model_2.predict(features_train_2))
predicted_valid_2 = pd.Series(model_2.predict(features_valid_2))

In [8]:
# Create a function to evaluate metrics MSE, RMSE, MAE and R2 Score
def eval_metrics(target_train, target_valid, predicted_valid):
    # Average Predicted Volume
    average_predicted_volume = predicted_valid.mean()
    average_valid_volume = target_valid.mean()
    print(f'Average Predicted Volume (model) = {average_predicted_volume.round(3)} thousand barrels')
    print(f'Average Actual Volume (validation) = {average_valid_volume.round(3)} thousand barrels')
    print()

    # Mean Squared Error
    mse = mean_squared_error(target_valid, predicted_valid)
    print('MSE (model) =', mse.round(3))
    predicted_mean = pd.Series(target_train.mean(), index=target_valid.index)
    mse_mean = mean_squared_error(target_valid, predicted_mean)
    print('MSE (mean) =', mse_mean.round(3))
    print()
    
    # Root Mean Squared Error
    rmse = mse ** 0.5
    print('RMSE (model) =', rmse.round(3))
    rmse_mean = mse_mean ** 0.5
    print('RMSE (mean) =', rmse_mean.round(3))
    print()
    
    # Mean Absolute Error (training, validation and median)
    mae = mean_absolute_error(target_valid, predicted_valid)
    print('MAE (model) =', mae.round(3))
    predicted_median = pd.Series(target_train.median(), index=target_valid.index)
    mae_median = mean_absolute_error(target_valid, predicted_median)
    print('MAE (median) =', mae_median.round(3))
    print()

    # R2 Score
    r2 = r2_score(target_valid, predicted_valid)
    print('R2 Score =', r2.round(3))
    print()
    
# Use function to evaluate metrics for each region
# REGION A
print('='*11, '\nREGION A\n', '='*11)
eval_metrics(target_train_0, target_valid_0, predicted_valid_0)

# REGION B
print('='*11, '\nREGION B\n', '='*11)
eval_metrics(target_train_1, target_valid_1, predicted_valid_1)

# REGION C
print('='*11, '\nREGION C\n', '='*11)
eval_metrics(target_train_2, target_valid_2, predicted_valid_2)

REGION A
Average Predicted Volume (model) = 92.593 thousand barrels
Average Actual Volume (validation) = 92.079 thousand barrels

MSE (model) = 1412.213
MSE (mean) = 1961.568

RMSE (model) = 37.579
RMSE (mean) = 44.29

MAE (model) = 30.92
MAE (median) = 37.672

R2 Score = 0.28

REGION B
Average Predicted Volume (model) = 68.729 thousand barrels
Average Actual Volume (validation) = 68.723 thousand barrels

MSE (model) = 0.798
MSE (mean) = 2117.973

RMSE (model) = 0.893
RMSE (mean) = 46.021

MAE (model) = 0.719
MAE (median) = 40.265

R2 Score = 1.0

REGION C
Average Predicted Volume (model) = 94.965 thousand barrels
Average Actual Volume (validation) = 94.884 thousand barrels

MSE (model) = 1602.378
MSE (mean) = 2016.221

RMSE (model) = 40.03
RMSE (mean) = 44.902

MAE (model) = 32.793
MAE (median) = 37.921

R2 Score = 0.205



The average volume of Reserves on Region A and C are higher than Region B, which suggests that these two regions are more productive.<br>

The MSE, RMSE and MAE metrics show that Linear Regression performs very well on Region B, and poorly on Region A and C.<br> 
However, these results seem unreliable, especially because a R2 score = 1.0 in Region B would indicate a perfect linear relation between the features and target, which is a quite rare situation.<br> 
We must further explore the data to try to find any inconsistencies such as data leakage, the presence of outliers, or if we need to pre-process the data before training the models.<br>

<div class="alert alert-success">
<b>Reviewer's comment</b>

The models were trained and evaluated correctly.
    
> However, these results seem unreliable, especially because a R2 score = 1.0 in Region B would indicate a perfect linear relation between the features and target, which is a quite rare situation.
    
Good point! :)

</div>

In [9]:
# Create a function to perform a Statistical Summary of the dataframes
def statistical_summary(features, target):
    # Combine features and target for a unified summary
    combined_data = pd.concat([features, target], axis=1)
    summary = combined_data.describe()
    print(summary)

# Print a Statistical Summary for each region and compare values:
print('='*60, '\nREGION A - Statistical Summary (Training set)\n', '='*60)
statistical_summary(features_train_0, target_train_0)
print('='*60, '\nREGION B - Statistical Summary (Training set)\n', '='*60)
statistical_summary(features_train_1, target_train_1)
print('='*60, '\nREGION C - Statistical Summary (Training set)\n', '='*60)
statistical_summary(features_train_2, target_train_2)

REGION A - Statistical Summary (Training set)
                 f0            f1            f2       product
count  75000.000000  75000.000000  75000.000000  75000.000000
mean       0.497441      0.250064      2.505917     92.640468
std        0.871824      0.504203      3.249684     44.288688
min       -1.408605     -0.848218    -10.138341      0.000000
25%       -0.075827     -0.199952      0.296284     56.652603
50%        0.499079      0.249690      2.519854     92.131131
75%        1.070328      0.700239      4.725356    128.719821
max        2.362331      1.343769     16.003790    185.364347
REGION B - Statistical Summary (Training set)
                 f0            f1            f2       product
count  75000.000000  75000.000000  75000.000000  75000.000000
mean       1.139877     -4.791885      2.495747     68.858955
std        8.965661      5.126975      1.702572     45.918736
min      -31.609576    -26.358598     -0.018144      0.000000
25%       -6.293124     -8.261050      1

At first sight, the marker values of the datasets seem to be within reasonable values.<br> 
The min and max values of the features f0 and f1 on Region B seem to be more extreme than in the other regions. This might be causing overfitting when training the model. We can use StandardScaler on this dataset and see if there is any difference in performance.

<div class="alert alert-warning">
<b>Reviewer's comment</b>

Not sure what overfitting has to do with it. Remember that overfitting is when a model learns to predict the train set targets very well, but fails to perform well on new data (e.g. validation set), we don't have that situation as our model performs great on validation data, right? 

</div>

In [10]:
# Apply StandardScaler on the Feature numeric columns for REGION B
# Define scaler variable
scaler = StandardScaler()
# Training set
scaler.fit(features_train_1)
features_train_scaled_1 = pd.DataFrame(scaler.transform(features_train_1),
                                       index=features_train_1.index, 
                                       columns=features_train_1.columns)

# Validation set
features_valid_scaled_1 = pd.DataFrame(scaler.transform(features_valid_1),
                                       index=features_valid_1.index, 
                                       columns=features_valid_1.columns)

print('='*50, '\nREGION B - Original Features (Training set)\n', '='*50)
display(features_train_1.head())
print('='*50, '\nREGION B - Scaled Features (Training set)\n', '='*50)
display(features_train_scaled_1.head())
print('='*50, '\nREGION B - Original Features (Validation set)\n', '='*50)
display(features_valid_1.head())
print('='*50, '\nREGION B - Scaled Features (Validation set)\n', '='*50)
display(features_valid_scaled_1.head())

# Train Linear Regression model for Scaled features on Region B
model_scaled_1 = LinearRegression()
model_scaled_1.fit(features_train_scaled_1, target_train_1)
predicted_train_scaled_1 = pd.Series(model_scaled_1.predict(features_train_scaled_1))
predicted_valid_scaled_1 = pd.Series(model_scaled_1.predict(features_valid_scaled_1))

# Compare Evaluation metrics between Original and Scaled Features
# REGION B (Original features)
print('='*11, '\nREGION B (Original Features)\n', '='*11)
eval_metrics(target_train_1, target_valid_1, predicted_valid_1)
# REGION B (Scaled features)
print('='*11, '\nREGION B (Scaled Features)\n', '='*11)
eval_metrics(target_train_1, target_valid_1, predicted_valid_scaled_1)

REGION B - Original Features (Training set)


,f0,f1,f2
27212,-6.488552,-1.590478,3.001311
7866,18.819463,4.602079,2.996867
62041,10.816499,-3.919653,1.991077
70185,-12.416362,-9.343774,0.996691
82230,-15.041012,-8.474624,1.996463


REGION B - Scaled Features (Training set)


,f0,f1,f2
27212,-0.850855,0.624428,0.296943
7866,1.971935,1.832275,0.294333
62041,1.079305,0.170127,-0.296418
70185,-1.512028,-0.887837,-0.880471
82230,-1.804775,-0.718311,-0.293255


REGION B - Original Features (Validation set)


,f0,f1,f2
71751,-0.371866,-1.862494,3.002210
80493,9.015122,-13.881455,1.995363
2655,-6.507568,-4.817448,1.003449
53233,14.560845,-10.667755,1.995175
91141,6.090476,-4.494723,0.013815


REGION B - Scaled Features (Validation set)


,f0,f1,f2
71751,-0.168616,0.571372,0.297471
80493,0.878384,-1.772903,-0.293901
2655,-0.852976,-0.004986,-0.876502
53233,1.496940,-1.146077,-0.294011
91141,0.552177,0.057961,-1.457764


REGION B (Original Features)
Average Predicted Volume (model) = 68.729 thousand barrels
Average Actual Volume (validation) = 68.723 thousand barrels

MSE (model) = 0.798
MSE (mean) = 2117.973

RMSE (model) = 0.893
RMSE (mean) = 46.021

MAE (model) = 0.719
MAE (median) = 40.265

R2 Score = 1.0

REGION B (Scaled Features)
Average Predicted Volume (model) = 68.729 thousand barrels
Average Actual Volume (validation) = 68.723 thousand barrels

MSE (model) = 0.798
MSE (mean) = 2117.973

RMSE (model) = 0.893
RMSE (mean) = 46.021

MAE (model) = 0.719
MAE (median) = 40.265

R2 Score = 1.0



After using StandardScaler on the Features, we see that the evaluation metrics are exactly the same. This is because in ordinary linear regression (without regularization), feature scaling doesn't affect the model's ability to fit the data or the evaluation metrics that measure the quality of the fit.<br> 
Therefore, we can keep the original features as they are and try to find another reason for the high R2 Score.

<div class="alert alert-success">
<b>Reviewer's comment</b>

You are right!

</div>

In [11]:
# Perform a Correlation Analysis on the Training sets
# Create a function to combine Features/Target and print Correlation Matrix
def find_corr(features, target, title):
    data = features.copy()
    data['product'] = target
    print('='*60)
    print(f'Correlation Matrix - {title}')
    print('='*60)
    print(data.corr())
    
# Print Correlation Matrix for Training sets on each Region
find_corr(features_train_0, target_train_0, 'Region A - Training set')
find_corr(features_train_1, target_train_1, 'Region B - Training set')
find_corr(features_train_2, target_train_2, 'Region C - Training set')

Correlation Matrix - Region A - Training set
               f0        f1        f2   product
f0       1.000000 -0.441091 -0.003445  0.139852
f1      -0.441091  1.000000  0.002173 -0.190642
f2      -0.003445  0.002173  1.000000  0.483181
product  0.139852 -0.190642  0.483181  1.000000
Correlation Matrix - Region B - Training set
               f0        f1        f2   product
f0       1.000000  0.182914 -0.003275 -0.032022
f1       0.182914  1.000000 -0.003612 -0.011246
f2      -0.003275 -0.003612  1.000000  0.999396
product -0.032022 -0.011246  0.999396  1.000000
Correlation Matrix - Region C - Training set
               f0        f1        f2   product
f0       1.000000  0.000198  0.000784  0.001546
f1       0.000198  1.000000 -0.000774 -0.001965
f2       0.000784 -0.000774  1.000000  0.443408
product  0.001546 -0.001965  0.443408  1.000000


We can see there is a near-perfect correlation (0.999396) between the **'f2'** Feature and the **'product'** Target on **Region B**. This coefficient indicates a direct linear correlation that accounts for the perfect **R2 score = 1.00** we found earlier. Altought this result suggests that 'f2' is a very reliable predictor of oil reserves, this strong direct correlation leads to **overfitting** when training the Linear Regression model, because the correlated feature overshadows the relations between the other features (f0, f1) and the target (multicollinearity). 

<div class="alert alert-warning">
<b>Reviewer's comment</b>

Again, unclear what you mean by overfitting here, I don't see any :)
    
There doesn't seem to be any multicollinearity as well: multicollinearity is when features are highly correlated with each other, not with target.
    
It might be the case of data leakage, but in this project f2 is indeed just a good predictor of the target in region B. But I commend your intuition on this: it is always a good idea to check for data leakage if the results look too good to be true! :)

<div class="alert alert-info">
  I see. Yeah, it seemed too good to be true!
</div>

In [12]:
# Exclude multicollinear features and train the model
features_train_temp_1 = features_train_1.drop('f2', axis= 1)
features_valid_temp_1 = features_valid_1.drop('f2', axis= 1)
print('='*50, "\nREGION B - Excluded 'f2' feature (Training set)\n", '='*50)
print(features_train_temp_1.head())
print('='*50, "\nREGION B - Excluded 'f2' feature (Validation set)\n", '='*50)
print(features_valid_temp_1.head())

# Train a Linear Regression model
model_temp_1 = LinearRegression()
model_temp_1.fit(features_train_temp_1, target_train_1)
predicted_train_temp_1 = pd.Series(model_temp_1.predict(features_train_temp_1))
predicted_valid_temp_1 = pd.Series(model_temp_1.predict(features_valid_temp_1))
print('='*50, "\nLinear Regression - Excluded 'f2' feature (Predictions)\n", '='*50)
print(predicted_valid_temp_1.head())

print('='*50, "\nREGION B - Excluded 'f2' feature (Evaluation metrics)\n", '='*50)
eval_metrics(target_train_1, target_valid_1, predicted_valid_temp_1)

REGION B - Excluded 'f2' feature (Training set)
              f0        f1
27212  -6.488552 -1.590478
7866   18.819463  4.602079
62041  10.816499 -3.919653
70185 -12.416362 -9.343774
82230 -15.041012 -8.474624
REGION B - Excluded 'f2' feature (Validation set)
              f0         f1
71751  -0.371866  -1.862494
80493   9.015122 -13.881455
2655   -6.507568  -4.817448
53233  14.560845 -10.667755
91141   6.090476  -4.494723
Linear Regression - Excluded 'f2' feature (Predictions)
0    68.952726
1    68.062351
2    70.074520
3    67.021316
4    68.058043
dtype: float64
REGION B - Excluded 'f2' feature (Evaluation metrics)
Average Predicted Volume (model) = 68.859 thousand barrels
Average Actual Volume (validation) = 68.723 thousand barrels

MSE (model) = 2116.64
MSE (mean) = 2117.973

RMSE (model) = 46.007
RMSE (mean) = 46.021

MAE (model) = 40.377
MAE (median) = 40.265

R2 Score = 0.001



If we exclude the feature 'f2', we encounter a significantly different scenario: an extremely poor performance of the Linear Regression model **(R2 = 0.001)**. This indicates there is **no linear correlation** between the **'f0, f1'** features and the **'product'** target. We face a challenging situation: the feature 'f2' shows a very strong positive linear correlation with the target, dominating the Linear Regression model's predictions; meanwhile, the other features 'f0, f1' either do not have a strong linear relationship with the target or their effects are too subtle to be captured by a Linear Regression model.<br>

In such cases, exploring the performance of a **Random Forest** model is advisable. Random Forest offers several advantages over Linear Regression, particularly in handling non-linear relationships and interactions between features that linear models may overlook.

We can evaluate the R2 score perfomance of the Validation set in several Random Forest models at different max_depths.

In [ ]:
# Create function to evaluate R2 Maximization using Random Forest
def evaluate_r2(features_train, target_train, features_valid, target_valid):
    for depth in range(1, 16, 1):
        model = RandomForestRegressor(n_estimators=100, max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        print(f'Max depth = {depth}')
        print('R2 value on training set =', model.score(features_train, target_train))
        print('R2 value on validation set =', model.score(features_valid, target_valid))

In [ ]:
%%time
# Evaluate the R2 perfomance of Random Forest model on each Region.
start_time = time.time()        
        
# REGION A
print('='*11, '\nREGION A\n', '='*11)
evaluate_r2(features_train_0, target_train_0, features_valid_0, target_valid_0)

end_time = time.time()
print(f"Execution time: {round(end_time - start_time, 3)} seconds")

In [ ]:
%%time
start_time = time.time()        

# REGION B
print('='*11, '\nREGION B\n', '='*11)
evaluate_r2(features_train_1, target_train_1, features_valid_1, target_valid_1)

end_time = time.time()
print(f"Execution time: {round(end_time - start_time, 3)} seconds")

In [ ]:
%%time
start_time = time.time()        

# REGION C
print('='*11, '\nREGION C\n', '='*11)
evaluate_r2(features_train_2, target_train_2, features_valid_2, target_valid_2)

end_time = time.time()
print(f"Execution time: {round(end_time - start_time, 3)} seconds")

The optimal parameters of the **Random Forest model (n_estimators = 100)** for each region are:<br>

**- REGION A**<br>
Max depth = 9<br>
R2 value on training set = 0.3290<br>
**R2 value on validation set = 0.2982**<br>

**- REGION B**<br>
Max depth = 7<br>
R2 value on training set = 0.9997<br>
**R2 value on validation set = 0.9997**<br>

**- REGION C**<br>
Max depth = 10<br>
R2 value on training set = 0.3422<br>
**R2 value on validation set = 0.2955**<br>

In [ ]:
# Calculate Metrics for the optimal configurations of Random Forest(n_estimators=100)

# REGION A
model_a0 = RandomForestRegressor(n_estimators=100, max_depth=9, random_state=12345)
model_a0.fit(features_train_0, target_train_0)
predicted_valid_a0 = model_a0.predict(features_valid_0)
print('='*80, "\nREGION A - Random Forest (n_estimators=100, max_depth=9) - (Evaluation metrics)\n", '='*80)
eval_metrics(target_train_0, target_valid_0, predicted_valid_a0)

# REGION B
model_a1 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=12345)
model_a1.fit(features_train_1, target_train_1)
predicted_valid_a1 = model_a1.predict(features_valid_1)
print('='*80, "\nREGION B - Random Forest (n_estimators=100, max_depth=7) - (Evaluation metrics)\n", '='*80)
eval_metrics(target_train_1, target_valid_1, predicted_valid_a1)

# REGION C
model_a2 = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=12345)
model_a2.fit(features_train_2, target_train_2)
predicted_valid_a2 = model_a2.predict(features_valid_2)
print('='*80, "\nREGION C - Random Forest (n_estimators=100, max_depth=10) - (Evaluation metrics)\n", '='*80)
eval_metrics(target_train_2, target_valid_2, predicted_valid_a2)


Using a Random Forest model with the optimal hyperparameters for each case, we see only a marginal improvement on metrics over Linear Regression:<br> 

**REGION A**<br> 
MSE (model) = 1412.213 to 1376.343<br> 
RMSE (model) = 37.579 to 37.099<br> 
MAE (model) = 30.92 to 30.524<br> 
R2 Score = 0.28 to 0.298<br> 

**REGION B**<br> 
MSE (model) = 0.798 to 0.515<br> 
RMSE (model) = 0.893 to 0.718<br> 
MAE (model) = 0.719 to 0.319<br> 
R2 Score = 1.0 to 1.0<br> 

**REGION C**<br> 
MSE (model) = 1602.378 to 1420.354<br> 
RMSE (model) = 40.03 to 37.688<br> 
MAE (model) = 32.793 to 31.11<br> 
R2 Score = 0.205 to 0.296<br> 

All these procedures have not taken us very far yet. Since Random Forest is not sufficiently predictable, we will stick to Linear Regression for Profit Calculation and prediction.

<div class="alert alert-success">
<b>Reviewer's comment</b>

Yeah, while this is an interesting experiment, the conditions of the task only allow linear regression unfortunately

<div class="alert alert-info">
    Ha yeah, I knew I was going on a unnecessary limb but I was curious to see how long would it take to run the loop!
</div>

## Profit Calculation

First we will define the key values for **profit calculation** and assign variables to them. We will then calculate the **Break-Even Volume per well**<br> 
that determines the profitability of the project and compare it with the average production in each region.<br> 
If the average volume per well of a given region is higher than the break-even volumen, we can say that the region is profitable.<br> 
We will then conduct an analysis using **Bootstrapping for Models** to calculate the total Revenue of each region for the given conditions. <br>

***Key Values for Profit Calculation:***<br>
Budget for Development: \\$100 million<br>
Number of Wells: 200<br>
Revenue ($4.5 per barrel x 1000): \\$4,500<br>

***Break-Even Volume of Reserves per Well:***<br>
This is the minimum volume needed for a well to be profitable under the given budget and revenue structure.<br>
It can be calculated by dividing the total budget for one region by the revenue per thousand barrels and then dividing by the number of wells.

<div class="alert alert-warning">
<b>Reviewer's comment</b>

To produce correct output when using \\$ symbol you need to escape it with backslashes like this: `\\$` produces \\$. Otherwise it acts as a delimiter of Latex formulas: $E = mc^2$

</div>

<div class="alert alert-info">
Thanks!
</div>

In [13]:
# 3.1 Store Key Values
BUDGET = 100_000_000  # USD for 200 wells
REVENUE_PER_BARREL = 4.5  # USD
REVENUE_PER_UNIT = 4500  # USD (since volume is in thousand barrels)
WELLS = 200

In [14]:
# 3.2 Calculate Break-Even Volume
break_even_volume_per_well = BUDGET / WELLS / REVENUE_PER_UNIT  # in thousand barrels
print(f"Break-even volume per well: {round(break_even_volume_per_well,3)} thousand barrels")
print()

# 3.3 Compare with Average Volume of Reserves
average_volume_0 = data_target_0.mean()
average_volume_1 = data_target_1.mean()
average_volume_2 = data_target_2.mean()
print(f"Average volume in Region A: {round(average_volume_0, 3)} thousand barrels")
print(f"Average volume in Region B: {round(average_volume_1, 3)} thousand barrels")
print(f"Average volume in Region C: {round(average_volume_2, 3)} thousand barrels")

Break-even volume per well: 111.111 thousand barrels

Average volume in Region A: 92.5 thousand barrels
Average volume in Region B: 68.825 thousand barrels
Average volume in Region C: 95.0 thousand barrels


All regions have a lower average volume per well than the minimum required for the region to be profitable (break-even).<br>
The highest average production per well is **Region C** with **95.0 average** thousand barrels per well.

<div class="alert alert-success">
<b>Reviewer's comment</b>

Right! So if we select the wells randomly, we're going to lose money on average. 

</div>

In [15]:
# Step 4: Write function to Calculate Profit
def calculate_profit(target, probabilities):
    # Reset the indices of target and probabilities and drop the old indices
    probabilities_reset = probabilities.reset_index(drop=True)
    target_reset = target.reset_index(drop=True)
    
    # Sort probabilities by values in descending order
    probs_sorted = probabilities_reset.sort_values(ascending=False)
    print(f"Predicted values sorted by descending order:\n{probs_sorted.sort_values(ascending=False).head()}")
    print(f"Shape of predicted probabilities: {probs_sorted.shape}\n")
    
    # Use sorted probabilities to select top 200 wells indices from target
    selected = target_reset.loc[probs_sorted.index][:WELLS]
    print(f"Top 200 Target values sorted by Index:\n{selected.sort_values(ascending=False).head()}")    
    print(f"Shape of selected targets: {selected.shape}\n")

    # Confirm that target and probabilities indices match
    matching_indices = probs_sorted.index.intersection(selected.index)
    print("Check matching indices in probabilities and target:\n", matching_indices)
    if len(matching_indices) == 200:
        print(f"There are {len(matching_indices)} matches. All the indices in predicted probabilities were found in the target.\n")
    else:
        print(f"There are {len(matching_indices)} matches. Not all the indices in predicted probabilities were found in the target.\n")

    # Calculate revenue and profit
    revenue = selected.sum() * REVENUE_PER_UNIT
    print("Revenue: ", round(revenue, 3))
    profit = revenue - BUDGET
    return profit
    
# Calculate Profit for each region
print(f"Region A - Total Profit = {calculate_profit(target_valid_0, predicted_valid_0)}\n")
print("="*60)
print(f"Region B - Total Profit = {calculate_profit(target_valid_1, predicted_valid_1)}\n")
print("="*60)
print(f"Region C - Total Profit = {calculate_profit(target_valid_2, predicted_valid_2)}\n")

Predicted values sorted by descending order:
9317     180.180713
219      176.252213
10015    175.850623
11584    175.658429
23388    173.299686
dtype: float64
Shape of predicted probabilities: (25000,)

Top 200 Target values sorted by Index:
19109    184.356455
6767     184.095785
12050    183.552159
15449    182.378042
3694     182.225128
Name: product, dtype: float64
Shape of selected targets: (200,)

Check matching indices in probabilities and target:
 Int64Index([ 9317,   219, 10015, 11584, 23388,  4296,  8993, 14125, 14707,
            12461,
            ...
             4495, 11157, 24747, 24923,  5572,  7888,  7890, 24051, 24160,
            20340],
           dtype='int64', length=200)
There are 200 matches. All the indices in predicted probabilities were found in the target.

Revenue:  133208260.431
Region A - Total Profit = 33208260.43139851

Predicted values sorted by descending order:
20430    139.818970
7777     139.773423
8755     139.703330
1178     139.560938
4285     

<div class="alert alert-danger">
<s><b>Reviewer's comment</b>

1. Profit is revenue minus cost :)
2. You need to take validation set targets, not all targets, and to make sure that the indices match, otherwise you're not selecting the corresponding targets for selected wells </s>
    
<div class="alert alert-info">
    Thank you. I think I got it right this time :)
</div>

<div class="alert alert-success">
<b>Reviewer's comment V2</b>

Yep, all good!

</div>

The **Expected Profit (Revenue – Budget)** from the Top 200 wells for each region is:<br>

Region A = \\$33,208,260.431<br>
Region B = \\$24,150,866.966<br>
Region C = \\$27,103,499.635<br>

The most profitable is **Region A** (\\$33.2M), followed by **Region C** (\\$27.1M) and **Region B** (\\$24.1M)

<div class="alert alert-warning">
<b>Reviewer's comment V2</b>

These are more like maximum profit values rather then expected (as they are calculated using overall top 200 wells in each region, and we're not actually selecting from all wells, but from a random sample of 500 wells which is highly unlikely to contain overall top 200)

</div>

To better assess the accuracy of expected profits, we can write a similar but more thoroughgoing Profit calculation function using the **Bootstrapping technique.**<br>
We will take 1000 random samples of 500 values (with replacement), and then use the indices of the top 200 predicted probabilities and select them from the target.<br>
This will allow us to calculate the **average Profit (mean)** within a **95% Confidence Interval**, indicating the **lower limit (2.5% quantile)**, **upper limit (97.5% quantile)** and **risk of loss percentage**.

In [19]:
# Set the display format to suppress scientific notation
pd.options.display.float_format = '{:,.3f}'.format

# Step 5: Write function to Calculate Risks and Profit Using Bootstrapping
def bootstrap_profit_calculation(target, probabilities):
    #Define Random State and create empty variable to store values
    state = np.random.RandomState(12345)
    values = []
    
    # Reset the indices of target and probabilities and drop the old indices
    probabilities_reset = probabilities.reset_index(drop=True)
    target_reset = target.reset_index(drop=True)
    
    # Start Bootstrapping loop for 1000 iterations
    for i in range(1000):
        # from probabilities, take samples (n=500) with replacement and sort values in descending order
        probs_sampled = probabilities_reset.sample(n=500, replace=True, random_state=state)
        probs_sampled = probs_sampled.sort_values(ascending=False)
        # Use sampled probabilities to select top 200 indices from target
        selected = target_reset.loc[probs_sampled.index][:WELLS]
        # Calculate profit for the selected indices
        revenue = selected.sum() * REVENUE_PER_UNIT
        profit = revenue - BUDGET
        # Append values to numpy array
        values.append(profit)
    
    # Convert array to pd Series and print to see the results
    values = pd.Series(values)
    print("Series of Expected Profit values from Bootstrapping:\n", values)
    
    # Calculate average profit and lower/upper limits (95% percentile)
    mean = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    
    # Calculate risk of losses and express it as a percentage
    losses = values[values < 0].count()
    risk_of_loss = (losses / 1000) * 100

    return mean, lower, upper, risk_of_loss

# Calculate profits, limits and risk of loss for each Region
print('='*10,'\nREGION A\n', '='*10)
mean, lower, upper, risk_of_loss = bootstrap_profit_calculation(target_valid_0, predicted_valid_0)
print(f"Average Profit (95% confidence interval): ${mean:,.3f} USD")
print(f"Lower limit (2.5% quantile): ${lower:,.3f} USD")
print(f"Upper limit (97.5% quantile): ${upper:,.3f} USD")
print(f"Risk of loss (percentage): {risk_of_loss}%")
print()
print('='*10,'\nREGION B\n', '='*10)
mean, lower, upper, risk_of_loss = bootstrap_profit_calculation(target_valid_1, predicted_valid_1)
print(f"Average Profit (95% confidence interval): ${mean:,.3f} USD")
print(f"Lower limit (2.5% quantile): ${lower:,.3f} USD")
print(f"Upper limit (97.5% quantile): ${upper:,.3f} USD")
print(f"Risk of loss (percentage): {risk_of_loss}%")
print()
print('='*10,'\nREGION C\n', '='*10)
mean, lower, upper, risk_of_loss = bootstrap_profit_calculation(target_valid_2, predicted_valid_2)
print(f"Average Profit (95% confidence interval): ${mean:,.3f} USD")
print(f"Lower limit (2.5% quantile): ${lower:,.3f} USD")
print(f"Upper limit (97.5% quantile): ${upper:,.3f} USD")
print(f"Risk of loss (percentage): {risk_of_loss}%")
print()

REGION A
Series of Expected Profit values from Bootstrapping:
 0     6,054,640.746
1     5,363,934.311
2     2,937,858.319
3     1,789,933.967
4     2,719,929.221
           ...     
995   5,253,551.075
996   7,790,093.982
997   6,494,122.266
998   3,149,995.064
999   2,197,183.636
Length: 1000, dtype: float64
Average Profit (95% confidence interval): $3,961,649.848 USD
Lower limit (2.5% quantile): $-1,112,155.459 USD
Upper limit (97.5% quantile): $9,097,669.416 USD
Risk of loss (percentage): 6.9%

REGION B
Series of Expected Profit values from Bootstrapping:
 0     2,280,161.635
1     3,343,156.783
2     2,537,046.946
3     6,139,661.282
4     3,571,430.171
           ...     
995   6,831,945.425
996   6,468,698.399
997   2,386,523.349
998   4,142,424.639
999   1,245,778.409
Length: 1000, dtype: float64
Average Profit (95% confidence interval): $4,560,451.058 USD
Lower limit (2.5% quantile): $338,205.094 USD
Upper limit (97.5% quantile): $8,522,894.539 USD
Risk of loss (percentage): 1

<div class="alert alert-danger">
<s><b>Reviewer's comment</b>

1. Same problem here: you need to take targets and predictions for the validation set, and their indices need to match, otherwise the rows won't have corresponding predictions and targets.
2. Also please use profit instead of revenue for average profit and 95% confidence interval bounds
3. Don't forget to calculate risk of losses (which can be calculated as the share of negative profits in the list of profits produced in bootstrapping

</div>

<div class="alert alert-success">
<b>Reviewer's comment V2</b>

All problems were fixed!

</div>

The values for each region in a 95% Confidence Interval using the Bootstrapping technique<br> 
(1000 iterations of the top 200 points in samples of 500) are:<br>

**REGION A:**<br>
Average Profit (95% confidence interval): \\$3,961,649.848 USD<br>
Lower limit (2.5% quantile): \\$-1,112,155.459 USD<br>
Upper limit (97.5% quantile): \\$9,097,669.416 USD<br>
Risk of loss (percentage): 6.9%<br>

**REGION B:**<br>
Average Profit (95% confidence interval): \\$4,560,451.058 USD<br>
Lower limit (2.5% quantile): \\$338,205.094 USD<br>
Upper limit (97.5% quantile): \\$8,522,894.539 USD<br>
Risk of loss (percentage): 1.5%<br>

**REGION C:**<br>
Average Profit (95% confidence interval): \\$4,044,038.666 USD<br>
Lower limit (2.5% quantile): \\$-1,633,504.134 USD<br>
Upper limit (97.5% quantile): \\$9,503,595.749 USD<br>
Risk of loss (percentage): 7.6%<br>

## CONCLUSION
Taking into account a development budget of \\$100M for 200 Oil Wells, and an average profit of \\$4,500 per oil well,<br>
the Bootstrapping analysis conclusively shows that **Region B** would be the most profitable project for several reasons:<br>
• Shows the **highest average profits** (\\$4.56M USD)<br>
• Shows the **lowest risk of loss** (1.5%)<br>
• Shows the **highest predictability**, due to a strong linear correlation between the 'f2' feature and the target.<br>

Both **Region A** and **Region C** have the disadvantage of showing a higher risk of loss and lower average profits than Region B.<br>
The risk of loss in these regions (under 10%) could still be considered as manageable if the potential profits were substantially higher than in Region B.<br>
However, this is not the case. These results lead us to conclude that it's not advisable to develop oil wells in Region A and Region C. Both regions present very similar case scenarios, altought if we had to make a choice between the two, we think that, in spite of its lower average profits, **Region A** is a slightly preferable situation due to a lower risk of loss (6.9%) and lower maximum losses (2.5% quantile = \\$-1.1M USD).<br>

In conclusion, we recommend **Region B** as a viable development project based on our analysis, which indicates a relatively low level of risk and higher average profits.

<div class="alert alert-danger">
<s><b>Reviewer's comment</b>

Please check the conclusions after fixing the problems above</s>

<div class="alert alert-info">
    Fixed. Thanks for the comments!
</div>

<div class="alert alert-success">
<b>Reviewer's comment V2</b>

You're welcome! Great work! The project is now accepted. Good luck on the next sprint! :)

</div>